In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import gzip

In [ ]:
books_path = 'data/goodreads_books.json.gz'
authors_path = 'data/goodreads_book_authors.json.gz'
genres_path = 'data/goodreads_book_genres_initial.json.gz'
interactions_path = 'data/goodreads_interactions.csv'
reviews_path = 'data/goodreads_reviews_spoiler_raw.json.gz'
chunksize = 100000

In [ ]:
books_df = pd.read_json(gzip.open(books_path), lines=True, chunksize=chunksize)
authors_df = pd.read_json(gzip.open(authors_path), lines=True, chunksize=chunksize)
genres_df = pd.read_json(gzip.open(genres_path), lines=True, chunksize=chunksize)
reviews_df = pd.read_json(gzip.open(reviews_path), lines=True, chunksize=chunksize)
int_df = pd.read_csv(interactions_path, chunksize=chunksize)

## Books

In [ ]:
books_counts = pd.DataFrame(columns=['text_reviews_count', 'ratings_count'])
for chunk in books_df:
    counts = chunk[['text_reviews_count', 'ratings_count']]
    books_counts = pd.concat([books_counts, counts])

In [ ]:
books_counts.describe()

In [ ]:
books_df = books_df.drop(
    columns=['series', 'asin', 'kindle_asin', 'similar_books', 'link', 'url', 'image_url',
             'edition_information', 'title_without_series', 'popular_shelves', 'publisher'])

def extract_authors(authors_dict):
    return [author['author_id'] for author in authors_dict]

authors_column = books_df['authors'].apply(extract_authors)
books_df['author_id'] = authors_column
books_df = books_df.explode('author_id').set_index('book_id')
books_df['author_id'] = books_df['author_id'].fillna(0).astype('int64')
books_df = books_df.join(authors_df, how='inner', on='author_id', lsuffix='_book', rsuffix='_author')

genres_df = genres_df['genres'].apply(ps.Series).join(genres_df)
genres_df = genres_df.drop(columns=['genres']).fillna(0)
genres_df = genres_df.set_index('book_id')
books_df = books_df.join(genres_df, how='inner', on='book_id')